In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import cv2
import numpy as np
import pandas as pd

# one person joints

#### smoking_sliding

In [66]:
n_joints = 36 # (x, y) X 16 = 32
n_frame = 32

In [67]:
df = pd.read_csv('/content/drive/MyDrive/Project/Project_Gotcha/ipynb/OpenPose/datasets/smoking_sliding.csv')

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,index
0,1001.31,191.794,974.681,265.363,915.879,268.263,886.412,365.387,892.256,453.755,1024.84,262.429,1054.22,347.831,1048.46,297.809,927.699,459.643,910.044,601.046,906.978,727.529,1001.36,459.647,998.307,600.927,980.710,721.544,992.316,177.022,1012.95,177.034,951.296,180.159,0.0,0.0,1
1,1001.37,191.830,974.702,265.326,918.839,268.218,886.498,365.393,892.316,456.577,1024.79,262.417,1057.15,344.984,1048.36,291.840,927.743,459.655,912.903,600.948,906.933,727.518,1001.35,459.628,998.328,600.859,983.550,718.776,992.381,177.024,1015.89,177.091,954.022,180.145,0.0,0.0,1
2,1004.18,194.697,974.788,265.375,921.714,268.241,886.477,365.502,892.311,456.764,1027.66,262.458,1068.83,344.883,1048.27,285.978,927.735,459.631,912.916,600.999,906.985,727.509,1001.35,459.597,998.339,600.916,983.544,718.800,995.287,177.095,1018.78,177.154,954.154,180.149,0.0,0.0,1
3,1004.25,194.760,977.661,262.410,924.738,265.354,889.402,362.514,889.416,453.677,1033.55,259.454,1068.98,344.923,1048.19,283.082,927.710,456.761,912.973,600.897,906.966,724.604,1004.12,456.782,998.337,600.847,983.553,721.557,995.361,177.167,1018.85,177.182,954.270,183.041,0.0,0.0,1
4,1007.08,194.796,977.692,262.430,927.524,265.432,889.363,362.596,889.320,453.680,1033.53,256.540,1071.88,344.884,1045.47,274.293,927.737,456.745,918.789,598.022,906.938,724.602,1004.11,456.753,998.381,598.095,980.696,721.574,998.112,177.201,1021.63,177.194,957.030,183.052,0.0,0.0,1


In [68]:
a = 0
b = a + n_frame
df = df.iloc[a : b]
df.reset_index(drop=True, inplace = True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,index
0,1001.31,191.794,974.681,265.363,915.879,268.263,886.412,365.387,892.256,453.755,1024.84,262.429,1054.22,347.831,1048.46,297.809,927.699,459.643,910.044,601.046,906.978,727.529,1001.36,459.647,998.307,600.927,980.710,721.544,992.316,177.022,1012.95,177.034,951.296,180.159,0.0,0.0,1
1,1001.37,191.830,974.702,265.326,918.839,268.218,886.498,365.393,892.316,456.577,1024.79,262.417,1057.15,344.984,1048.36,291.840,927.743,459.655,912.903,600.948,906.933,727.518,1001.35,459.628,998.328,600.859,983.550,718.776,992.381,177.024,1015.89,177.091,954.022,180.145,0.0,0.0,1
2,1004.18,194.697,974.788,265.375,921.714,268.241,886.477,365.502,892.311,456.764,1027.66,262.458,1068.83,344.883,1048.27,285.978,927.735,459.631,912.916,600.999,906.985,727.509,1001.35,459.597,998.339,600.916,983.544,718.800,995.287,177.095,1018.78,177.154,954.154,180.149,0.0,0.0,1
3,1004.25,194.760,977.661,262.410,924.738,265.354,889.402,362.514,889.416,453.677,1033.55,259.454,1068.98,344.923,1048.19,283.082,927.710,456.761,912.973,600.897,906.966,724.604,1004.12,456.782,998.337,600.847,983.553,721.557,995.361,177.167,1018.85,177.182,954.270,183.041,0.0,0.0,1
4,1007.08,194.796,977.692,262.430,927.524,265.432,889.363,362.596,889.320,453.680,1033.53,256.540,1071.88,344.884,1045.47,274.293,927.737,456.745,918.789,598.022,906.938,724.602,1004.11,456.753,998.381,598.095,980.696,721.574,998.112,177.201,1021.63,177.194,957.030,183.052,0.0,0.0,1


In [69]:
for i in range(n_frame):
    if i % 2 == 0:
        df[str(i)] = df[str(i)] / 1920.0 * 640.0
    else:
        df[str(i)] = df[str(i)] / 1080.0 * 480.0

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,index
0,333.770000,85.241778,324.893667,117.939111,305.293000,119.228000,295.470667,162.394222,297.418667,201.668889,341.613333,116.635111,351.406667,154.591556,349.486667,132.359556,309.233000,204.285778,303.348000,267.131556,302.326000,323.346222,333.786667,204.287556,332.769000,267.078667,326.903333,320.686222,330.772000,78.676444,337.650000,78.681778,951.296,180.159,0.0,0.0,1
1,333.790000,85.257778,324.900667,117.922667,306.279667,119.208000,295.499333,162.396889,297.438667,202.923111,341.596667,116.629778,352.383333,153.326222,349.453333,129.706667,309.247667,204.291111,304.301000,267.088000,302.311000,323.341333,333.783333,204.279111,332.776000,267.048444,327.850000,319.456000,330.793667,78.677333,338.630000,78.707111,954.022,180.145,0.0,0.0,1
2,334.726667,86.532000,324.929333,117.944444,307.238000,119.218222,295.492333,162.445333,297.437000,203.006222,342.553333,116.648000,356.276667,153.281333,349.423333,127.101333,309.245000,204.280444,304.305333,267.110667,302.328333,323.337333,333.783333,204.265333,332.779667,267.073778,327.848000,319.466667,331.762333,78.708889,339.593333,78.735111,954.154,180.149,0.0,0.0,1
3,334.750000,86.560000,325.887000,116.626667,308.246000,117.935111,296.467333,161.117333,296.472000,201.634222,344.516667,115.312889,356.326667,153.299111,349.396667,125.814222,309.236667,203.004889,304.324333,267.065333,302.322000,322.046222,334.706667,203.014222,332.779000,267.043111,327.851000,320.692000,331.787000,78.740889,339.616667,78.747556,954.270,183.041,0.0,0.0,1
4,335.693333,86.576000,325.897333,116.635556,309.174667,117.969778,296.454333,161.153778,296.440000,201.635556,344.510000,114.017778,357.293333,153.281778,348.490000,121.908000,309.245667,202.997778,306.263000,265.787556,302.312667,322.045333,334.703333,203.001333,332.793667,265.820000,326.898667,320.699556,332.704000,78.756000,340.543333,78.752889,957.030,183.052,0.0,0.0,1


In [70]:
col = [str(i) for i in range(n_joints)]

arr = [[] * i for i in range(n_frame)]

for i in range(n_frame - 1):
    for c in col:
        arr[i].append(df[c][i])

In [71]:
len(arr)

32

In [72]:
from google.colab.patches import cv2_imshow
import os


if not os.path.exists('Videos'):
    os.mkdir('Videos')

if not os.path.exists('Images'):
    os.mkdir('Images')


size = (480, 640)
fps = 30
img_arr = []


for i in range(n_frame - 1):
    img = np.zeros(size + (3,))

    for idx, xy in enumerate(arr[i]):
        if (idx + 1)% 2 == 0 :
            x,y = (np.float32(arr[i][idx - 1]), np.float32(arr[i][idx]))
            cv2.circle(img, (x,y), 1, (0,255,0), cv2.LINE_4) # image, (x,y), radius, color)


    img_path = './Images/img_' + str('%02d' %i) + '.jpg'
    cv2.imwrite(img_path, img)

In [73]:
!rm ./test.mp4

!ffmpeg -f image2 -r 30 -i ./Images/img_%02d.jpg -vcodec libx264 ./test.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [74]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('test.mp4', width=640, height=480)

#### punch_sliding


In [75]:
n_joints = 36 # (x, y) X 18 = 36
n_frame = 32

In [76]:
df2 = pd.read_csv('/content/drive/MyDrive/Project/Project_Gotcha/ipynb/OpenPose/datasets/punch_sliding.csv')

df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,y
0,290.693,161.699,307.602,207.250,285.384,205.939,275.033,250.267,265.938,286.746,331.090,211.121,337.569,259.385,327.208,295.910,286.718,291.991,291.966,354.547,289.371,401.517,318.035,294.563,328.432,358.453,337.620,419.732,290.661,158.996,298.499,160.267,0.0,0.0,317.984,164.255,2
1,290.692,161.703,307.593,205.975,284.154,204.656,275.023,250.258,265.927,286.758,331.096,211.115,337.552,263.272,325.892,298.512,285.515,291.992,291.964,354.542,289.346,401.530,318.033,294.569,328.429,358.463,337.624,420.964,290.662,159.004,298.495,160.273,0.0,0.0,317.984,164.251,2
2,291.954,161.696,307.587,207.241,284.136,204.675,275.016,251.537,265.938,286.740,331.109,211.125,337.554,263.290,325.854,298.535,285.498,291.992,291.975,354.535,289.367,401.528,318.019,294.574,328.420,358.453,337.624,420.957,290.669,158.995,298.510,160.266,0.0,0.0,317.990,164.251,2
3,290.694,161.694,307.602,207.250,284.168,205.932,275.020,251.551,265.923,286.781,331.107,211.127,337.545,263.302,325.848,299.779,285.515,293.261,291.966,354.561,289.356,401.517,318.030,294.579,328.419,358.455,337.629,420.955,290.658,158.992,298.504,160.261,0.0,0.0,316.802,164.252,2
4,290.694,161.683,307.600,207.254,284.173,205.931,275.016,251.550,265.919,286.792,331.101,211.132,337.561,263.289,325.891,299.781,286.713,293.268,291.954,354.529,289.358,401.508,318.028,294.586,328.429,358.462,337.632,420.970,290.667,158.981,298.509,160.255,0.0,0.0,317.985,164.270,2


In [77]:
a = 0
b = a + n_frame
df2 = df2.iloc[a : b]
df2.reset_index(drop=True, inplace = True)
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,y
0,290.693,161.699,307.602,207.250,285.384,205.939,275.033,250.267,265.938,286.746,331.090,211.121,337.569,259.385,327.208,295.910,286.718,291.991,291.966,354.547,289.371,401.517,318.035,294.563,328.432,358.453,337.620,419.732,290.661,158.996,298.499,160.267,0.0,0.0,317.984,164.255,2
1,290.692,161.703,307.593,205.975,284.154,204.656,275.023,250.258,265.927,286.758,331.096,211.115,337.552,263.272,325.892,298.512,285.515,291.992,291.964,354.542,289.346,401.530,318.033,294.569,328.429,358.463,337.624,420.964,290.662,159.004,298.495,160.273,0.0,0.0,317.984,164.251,2
2,291.954,161.696,307.587,207.241,284.136,204.675,275.016,251.537,265.938,286.740,331.109,211.125,337.554,263.290,325.854,298.535,285.498,291.992,291.975,354.535,289.367,401.528,318.019,294.574,328.420,358.453,337.624,420.957,290.669,158.995,298.510,160.266,0.0,0.0,317.990,164.251,2
3,290.694,161.694,307.602,207.250,284.168,205.932,275.020,251.551,265.923,286.781,331.107,211.127,337.545,263.302,325.848,299.779,285.515,293.261,291.966,354.561,289.356,401.517,318.030,294.579,328.419,358.455,337.629,420.955,290.658,158.992,298.504,160.261,0.0,0.0,316.802,164.252,2
4,290.694,161.683,307.600,207.254,284.173,205.931,275.016,251.550,265.919,286.792,331.101,211.132,337.561,263.289,325.891,299.781,286.713,293.268,291.954,354.529,289.358,401.508,318.028,294.586,328.429,358.462,337.632,420.970,290.667,158.981,298.509,160.255,0.0,0.0,317.985,164.270,2


In [78]:
col = [str(i) for i in range(n_joints)]

arr = [[] * i for i in range(n_frame)]

for i in range(n_frame - 1):
    for c in col:
        arr[i].append(df2[c][i])

In [79]:
from google.colab.patches import cv2_imshow
import os


if not os.path.exists('Videos'):
    os.mkdir('Videos')

if not os.path.exists('Images'):
    os.mkdir('Images')


size = (480, 640)
fps = 30
img_arr = []


for i in range(n_frame - 1):
    img = np.zeros(size + (3,))

    for idx, xy in enumerate(arr[i]):
        if (idx + 1)% 2 == 0 :
            x,y = (np.float32(arr[i][idx - 1]), np.float32(arr[i][idx]))
            cv2.circle(img, (x,y), 1, (0,255,0), cv2.LINE_4) # image, (x,y), radius, color)


    img_path = './Images/img2_' + str('%02d' %i) + '.jpg'
    cv2.imwrite(img_path, img)

In [80]:
!rm ./test2.mp4

!ffmpeg -f image2 -r 30 -i ./Images/img2_%02d.jpg -vcodec libx264 ./test2.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [81]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('test2.mp4', width=640, height=480)

# two people joints

In [130]:
df = pd.read_csv('/content/drive/MyDrive/Project/Project_Gotcha/ipynb/datasets/X_joint_data.csv')
df

,index,p_1,p_2
0,assault-falldown0,"380.343, 134.948, 358.107, 162.386, 355.5, 163...","495.219, 138.845, 517.383, 163.653, 514.745, 1..."
1,assault-falldown0,"380.305, 137.513, 358.076, 162.39, 352.898, 16...","487.311, 137.525, 509.543, 162.376, 505.647, 1..."
2,assault-falldown0,"380.285, 134.96, 356.785, 162.383, 351.604, 16...","484.767, 136.251, 506.958, 162.33, 505.55, 162..."
3,assault-falldown0,"380.254, 134.954, 354.209, 162.368, 350.323, 1...","482.122, 134.921, 505.651, 162.403, 502.978, 1..."
4,assault-falldown0,"376.403, 138.798, 351.592, 162.372, 350.262, 1...","474.302, 137.56, 497.76, 163.622, 495.174, 162..."
...,...,...,...
7043,smoke-stand9,"522.551, 144.064, 536.966, 181.901, 517.351, 1...","298.079, 161.022, 282.402, 193.69, 275.852, 19..."
7044,smoke-stand9,"521.286, 144.074, 536.962, 181.909, 517.342, 1...","298.08, 161.009, 281.104, 193.665, 274.604, 19..."
7045,smoke-stand9,"522.551, 144.066, 536.961, 181.906, 517.345, 1...","298.093, 161.012, 281.117, 193.689, 274.555, 1..."
7046,smoke-stand9,"522.55, 144.062, 536.961, 181.908, 517.343, 18...","298.119, 161.01, 281.093, 193.652, 273.267, 19..."


In [133]:
!rm ./test.mp4
!ffmpeg -f image2 -r 30 -i ./Images/img_%02d.jpg -vcodec libx264 ./test.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [134]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('test.mp4', width=640, height=480)